In [1]:
#!pip install .
!pip show mosaic_llm

Name: mosaic_llm
Version: 0.1.4
Summary: The main goal of this code is to use Zero-shot LLM to improve/Expand user query, fetch search results using MOSAIC RESTAPI and then summarize the snippets using also Zero-shot prompting.
Home-page: 
Author: 
Author-email: Roxanne El Baff <roxanne.elbaff@gmail.com>, Valentin <v.edelsbrunner@protonmail.com>
License: 
Location: C:\Users\elba_ro\.pyenv\pyenv-win\versions\3.11.2\Lib\site-packages
Requires: langchain, langchain-mistralai, langchain_community, pandas, python-dotenv, requests, tqdm
Required-by: 


In [2]:
from  mosaic_llm.mosaicllm import MosaicLLM  


In [6]:
# SAMPLE RUN

q= "Apple"
mosaicllm = MosaicLLM(root="")
apple_query_result = mosaicllm.run(query=q)

print("original query summary:")
print(apple_query_result['summary_query'])

print("\nimproved query summary:")
print(apple_query_result['summary_clarified_query'])

print("\nsubquery_1-3 summary:")
print(apple_query_result['summary_subqueries'])

original query summary:
The snippet mentions Apple in the context of its annual Worldwide Developers Conference (WWDC), where the company showcased new features for its operating systems, including iOS 16, iPadOS 16, macOS 13, and watchOS 9. The updates include improvements to the lock screen, notification system, and health tracking capabilities, as well as new features for developers. The conference also featured updates on Apple's augmented reality and virtual reality initiatives, including the announcement of the realityOS, a new operating system for AR/VR headsets.

improved query summary:
The text snippets provide information about Apple Inc., a multinational technology company headquartered in Cupertino, California. The company is known for designing, manufacturing, and marketing consumer electronics, computer software, and online services. Its most popular products include the iPhone smartphone, the iPad tablet computer, the Mac personal computer, the iPod portable media player

In [7]:
apple_query_result

{'rationale': "The initial query is just 'Apple' which is ambiguous. It could mean the fruit, the tech company, or even a color. I will clarify the query to 'Apple company' and the sub-queries would include queries about the company's products, headquarters, and CEO.",
 'clarified_query': 'Apple company',
 'subqueries': ['Apple products', 'Apple headquarters', 'Apple CEO'],
 'prompt_query_summary': 'Human: ### INSTRUCTIONS\n\nWe will provide you with a search query and a list of the result snippets. \n\nYour task is to Summarize the result snippets in one paragraph, FOCUSING on "Apple"\n\n### OUTPUT\n\nOutput only your SUMMARY without leading or trailing sentences and without any explanation.\n\n# INPUT \n\nFOCUS: Apple\n\nList of Text snippets to summarize:\n\n- \n',
 'summary_query': "The snippet mentions Apple in the context of its annual Worldwide Developers Conference (WWDC), where the company showcased new features for its operating systems, including iOS 16, iPadOS 16, macOS 13,

In [3]:
import os.path
import pandas as pd

def run_all_queries():
    queries = pd.read_csv("data/queries.csv", encoding="utf8")
    all_results=[]
    out = "data/mosaicllm_results.csv"
    if os.path.exists(out):
        return pd.read_csv(out)
    mosaicllm = MosaicLLM(root="")
    for _, row in queries.iterrows():
        result = {}
        try:
            q = row["query"]
            print(f"querying {q}")
            result = mosaicllm.run(query=q)
            result["query"] = q
            all_results.append(result)
            
        except Exception as e:
            print(f"exception in {e} for {q}")
        break
    df = pd.DataFrame(all_results)

    if len(df)>0:
        df.to_csv(out)
    return df

df = run_all_queries()

querying Apple
HTTP error occurred: 502 Server Error: Proxy Error for url: https://qnode.eu/ows/mosaic/service/search?q=Apple+Inc.+headquarters?index=demo-simplewiki&lang=en&limit=5&index=demo-simplewiki&lang=en&limit=5
exception in string indices must be integers, not 'str' for Apple


In [10]:
df

""
